In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import VotingClassifier
#Stop warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.read_csv('Pokemon.csv')

In [4]:
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
data = data.set_index('#')
data.head()
#Type1 is our response variable we wish to classify
type1 = data.pop('Type 1')

In [6]:
data.head()

,Name,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,NaN,309,39,52,43,60,50,65,1,False


# Data Processing

In [8]:
type1.value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Ground       32
Ghost        32
Dragon       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [8]:
# create a binary response variable for each class
Y = pd.get_dummies(type1)
np.sum(Y, axis=0)

Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
dtype: int64

In [10]:
#What response variable currently looks like
Y.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
#,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [11]:
dummies2list = []
for x in Y:
    word = 'Type_1_' + x
    dummies2list.append(word)
Y.columns = dummies2list

In [12]:
#Allows us to distinguish between type 1 and type 2 later on
Y.head()

,Type_1_Bug,Type_1_Dark,Type_1_Dragon,Type_1_Electric,Type_1_Fairy,Type_1_Fighting,Type_1_Fire,Type_1_Flying,Type_1_Ghost,Type_1_Grass,Type_1_Ground,Type_1_Ice,Type_1_Normal,Type_1_Poison,Type_1_Psychic,Type_1_Rock,Type_1_Steel,Type_1_Water
#,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [14]:
#Processes the generation column and type2 column by converting them into binary variables
dummies = pd.get_dummies(data['Generation'])
dummies3 = pd.get_dummies(data['Type 2'])

dummies1list = []
for x in dummies:
    word = 'Generation_' + str(x)
    dummies1list.append(word)
    
dummies.columns = dummies1list
step_1 = pd.concat([data, dummies], axis=1)  

dummies3list = []
for x in dummies3:
    word = 'Type_2_' + x
    dummies3list.append(word)
dummies3.columns = dummies3list
final_step = pd.concat([step_1,dummies3], axis = 1)

del final_step['Type 2']
del final_step['Generation']
del final_step['Name']
final_step

#Converts the boolean column into a column of 0's and 1's
final_step.Legendary = final_step.Legendary.astype(int)

In [15]:
final_step.head()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Generation_1,Generation_2,...,Type_2_Ghost,Type_2_Grass,Type_2_Ground,Type_2_Ice,Type_2_Normal,Type_2_Poison,Type_2_Psychic,Type_2_Rock,Type_2_Steel,Type_2_Water
#,,,,,,,,,,,,,,,,,,,,,
1,318,45,49,49,65,65,45,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,405,60,62,63,80,80,60,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,525,80,82,83,100,100,80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,625,80,100,123,122,120,80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,309,39,52,43,60,50,65,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
final_step.columns

Index(['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Legendary', 'Generation_1', 'Generation_2', 'Generation_3',
       'Generation_4', 'Generation_5', 'Generation_6', 'Type_2_Bug',
       'Type_2_Dark', 'Type_2_Dragon', 'Type_2_Electric', 'Type_2_Fairy',
       'Type_2_Fighting', 'Type_2_Fire', 'Type_2_Flying', 'Type_2_Ghost',
       'Type_2_Grass', 'Type_2_Ground', 'Type_2_Ice', 'Type_2_Normal',
       'Type_2_Poison', 'Type_2_Psychic', 'Type_2_Rock', 'Type_2_Steel',
       'Type_2_Water'],
      dtype='object')

# Modelling

In [21]:
#Creating training and validation set
X_train, X_test, y_train, y_test = train_test_split( final_step, Y, test_size = 0.3, random_state = 100)

#Normalise the features for some models
norm_train = preprocessing.normalize(X_train)
norm_test = preprocessing.normalize(X_test)

In [34]:
#Computes accuracy and confusion matrices
#Not normalising features
def predictingModel(model):
    
    accuracy = []
    confusion_matrices = []
    
    for y_type in y_train:
        model.fit(X_train, (y_train[y_type]))
        pred = model.predict(X_test)
        confusion_matrices.append(confusion_matrix(y_test[y_type],pred))
        accuracy.append(accuracy_score(y_test[y_type],pred))
    return accuracy, confusion_matrices

In [35]:
#For models where we need to normalise features
def predictingModel2(model):
    
    accuracy = []
    confusion_matrices = []
    
    for y_type in y_train:
        model.fit(norm_train, (y_train[y_type]))
        pred = model.predict(norm_test)
        confusion_matrices.append(confusion_matrix(y_test[y_type],pred))
        accuracy.append(accuracy_score(y_test[y_type],pred))
    return accuracy, confusion_matrices

#### Models to be used: Naive Bayes, Linear Discriminant Analysis, Quadratic Discriminant Analysis, Logistic Regression, Decision Tree, Random Forest, Extremely Random Forest, Adaptive Boost, Gradient Boosting

##### Naive Bayes

In [110]:
#Naive Bayes
NaiveB = GaussianNB()
NB_A, NB_C = predictingModel2(NaiveB)

##### Linear Discriminant Analysis

In [38]:
#Linear Discriminant analysis
LDA = LinearDiscriminantAnalysis()
LDA_A, LDA_C = predictingModel2(LDA)

##### Quadratic Discriminant Analysis

In [39]:
#Quadratic Discriminant analysis
QDA = QuadraticDiscriminantAnalysis()
QDA_A, QDA_C = predictingModel2(QDA)

##### Logistic Regression

In [40]:
#Logistic regression
LogReg = LogisticRegression(random_state=450411920)
LG_A, LG_C = predictingModel(LogReg)

##### Decision Tree

For the subsequent models, we have hyperparameters to tune. Therefore we use randomized search instead of grid search in order to save on computation time. This tries out random hyperparameters instead of all possible combination of hyperparameters, dramatically cutting down computation.

In [89]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [5, None],
              "max_features": range(1, 9),
              "min_samples_leaf": range(1, 9),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [94]:
DT_A, DT_C = predictingModel(tree_cv)

##### Random Forest

In [99]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": range(5,15),
              "max_depth": [3, None],
              "max_features": range(1, 9),
              "min_samples_split": range(2,5),
              "min_samples_leaf": range(1, 9),
              }

# Instantiate a Extremely Random Forest classifier: randext
randFor = RandomForestClassifier()

# Instantiate the RandomizedSearchCV object: randFor_cv
randFor_cv = RandomizedSearchCV(randFor,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [100]:
RF_A, RF_C = predictingModel(randFor_cv)

##### Extremely Random Forest

In [101]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": range(5,15),
              "max_depth": [3, None],
              "max_features": range(1, 9),
              "min_samples_split": range(2,5),
              "min_samples_leaf": range(1, 9),
              }

# Instantiate a Extremely Random Forest classifier: randext
randExt = ExtraTreesClassifier()

# Instantiate the RandomizedSearchCV object: randext_cv
randExt_cv = RandomizedSearchCV(randExt,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [102]:
ERF_A, ERF_C = predictingModel(randExt_cv)

##### Adaptive Boosting

In [103]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": range(330,400,10),
              "learning_rate": range(1,5),
              }

# Instantiate a Adaptie boosting classifier: tree
randAda = AdaBoostClassifier()

# Instantiate the RandomizedSearchCV object: randada_cv
randAda_cv = RandomizedSearchCV(randAda,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [104]:
AB_A, AB_C = predictingModel(randAda_cv)

##### Gradient Boosting

In [105]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"loss": ["deviance", "exponential"],
              "n_estimators": range(250,550,50),
              "learning_rate": [0.1],
              "min_samples_split" : [2,4],
              "min_samples_leaf" : range(1,3)
              }

# Instantiate a GB boosting classifier: randGB
randGB = GradientBoostingClassifier()

# Instantiate the RandomizedSearchCV object: randGB_cv
randGB_cv = RandomizedSearchCV(randGB,param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)

In [106]:
GB_A, GB_C = predictingModel(randGB_cv)

Looking at how our models performed so far:

In [160]:
models = ['Naive Bayes', 'LDA', 'QDA', 'Logistic Regression','Decision Tree','Random Forest','Extremely Random Forest','Adaptive Boosting','Gradient Boosting']

In [161]:
accuracy_scores = [NB_A, LDA_A, QDA_A, LG_A,DT_A, RF_A, ERF_A, AB_A, GB_A]

In [163]:
accuracy = pd.DataFrame(data = accuracy_scores).transpose()
accuracy.columns = models
accuracy.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting
Type_1_Bug,0.325,0.917,0.329,0.938,0.946,0.933,0.925,0.917,0.921
Type_1_Dark,0.492,0.925,0.950,0.950,0.950,0.950,0.954,0.933,0.946
Type_1_Dragon,0.575,0.871,0.954,0.954,0.950,0.933,0.954,0.938,0.946
Type_1_Electric,0.333,0.925,0.354,0.929,0.929,0.938,0.929,0.921,0.946
Type_1_Fairy,0.846,0.962,0.967,0.967,0.967,0.967,0.967,0.958,0.962
Type_1_Fighting,0.383,0.954,0.979,0.962,0.971,0.971,0.971,0.929,0.962
Type_1_Fire,0.233,0.946,0.233,0.946,0.921,0.942,0.946,0.912,0.929
Type_1_Flying,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988
Type_1_Ghost,0.312,0.967,0.496,0.979,0.975,0.983,0.983,0.975,0.983
Type_1_Grass,0.233,0.912,0.212,0.933,0.938,0.921,0.929,0.896,0.929


In [168]:
accuracy.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting
count,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000
mean,0.426,0.927,0.671,0.942,0.940,0.942,0.944,0.923,0.938
std,0.197,0.035,0.333,0.035,0.034,0.033,0.032,0.045,0.035
min,0.221,0.850,0.192,0.846,0.862,0.867,0.871,0.792,0.838
25%,0.314,0.918,0.335,0.933,0.923,0.930,0.929,0.906,0.929
50%,0.358,0.929,0.906,0.952,0.948,0.946,0.954,0.929,0.946
75%,0.483,0.949,0.957,0.961,0.964,0.966,0.965,0.949,0.950
max,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988


In [169]:
for i,j in zip(models,accuracy_scores):
    print(i,' average accuracy is: ', np.average(j).round(3))

Naive Bayes  average accuracy is:  0.426
LDA  average accuracy is:  0.927
QDA  average accuracy is:  0.671
Logistic Regression  average accuracy is:  0.942
Decision Tree  average accuracy is:  0.94
Random Forest  average accuracy is:  0.942
Extremely Random Forest  average accuracy is:  0.944
Adaptive Boosting  average accuracy is:  0.923
Gradient Boosting  average accuracy is:  0.938


### Ensemble Creation

From this, we create a final ensemble by combining the top 3 uncorrelated models and take a soft vote (by assigning a higher weight to the vote for a "better" model) on which class does an observation belong to. The top uncorrelated models are:
- Extremely Random Forest

- Logistic Regression

- Gradient Boosting

In [185]:
finalensem = VotingClassifier(estimators=[('Ext Rand Forest', randExt_cv), ('Log Reg', LogReg), ('GradientBoost', randGB_cv)], voting='soft')

In [186]:
accuracy_finalensem = []
for y_type in y_train:
    finalensem.fit(X_train, y_train[y_type])
    pred = finalensem.predict(X_test)
    accuracy_finalensem.append(accuracy_score(y_test[y_type],pred))

In [187]:
models.append('Final Ensemble')
accuracy_scores.append(accuracy_finalensem)

In [188]:
accuracy2 = pd.DataFrame(data = accuracy_scores).transpose()
accuracy2.columns = models
accuracy2.set_index(Y.columns).round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
Type_1_Bug,0.325,0.917,0.329,0.938,0.946,0.933,0.925,0.917,0.921,0.933
Type_1_Dark,0.492,0.925,0.950,0.950,0.950,0.950,0.954,0.933,0.946,0.950
Type_1_Dragon,0.575,0.871,0.954,0.954,0.950,0.933,0.954,0.938,0.946,0.954
Type_1_Electric,0.333,0.925,0.354,0.929,0.929,0.938,0.929,0.921,0.946,0.938
Type_1_Fairy,0.846,0.962,0.967,0.967,0.967,0.967,0.967,0.958,0.962,0.967
Type_1_Fighting,0.383,0.954,0.979,0.962,0.971,0.971,0.971,0.929,0.962,0.971
Type_1_Fire,0.233,0.946,0.233,0.946,0.921,0.942,0.946,0.912,0.929,0.946
Type_1_Flying,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988,0.992
Type_1_Ghost,0.312,0.967,0.496,0.979,0.975,0.983,0.983,0.975,0.983,0.983
Type_1_Grass,0.233,0.912,0.212,0.933,0.938,0.921,0.929,0.896,0.929,0.933


In [190]:
accuracy2.describe().round(3)

,Naive Bayes,LDA,QDA,Logistic Regression,Decision Tree,Random Forest,Extremely Random Forest,Adaptive Boosting,Gradient Boosting,Final Ensemble
count,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000,18.000
mean,0.426,0.927,0.671,0.942,0.940,0.942,0.944,0.923,0.938,0.945
std,0.197,0.035,0.333,0.035,0.034,0.033,0.032,0.045,0.035,0.032
min,0.221,0.850,0.192,0.846,0.862,0.867,0.871,0.792,0.838,0.867
25%,0.314,0.918,0.335,0.933,0.923,0.930,0.929,0.906,0.929,0.934
50%,0.358,0.929,0.906,0.952,0.948,0.946,0.954,0.929,0.946,0.952
75%,0.483,0.949,0.957,0.961,0.964,0.966,0.965,0.949,0.950,0.961
max,0.942,0.975,0.992,0.992,0.992,0.992,0.992,0.992,0.988,0.992


Conclusively, we have an average accuracy of 94.5% for our final ensemble, an improvement from the 3 base models we included in our ensemble.